# 1. Install and Import Dependencies

In [1]:
!pip install mediapipe opencv-python


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import mediapipe as mp
import cv2
import numpy as np
import uuid
import os

In [3]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

# 2. Take Output Images 

In [4]:
import cv2
import mediapipe as mp
import os
import time

# Initialisation of MediaPipe and Hands
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

# do not modify parameters
hands = mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5) 

# set category of current images
# {zero, one, two, three, four, five, spock}
imageCategory = "other"

# folder for images
os.makedirs(imageCategory, exist_ok=True) 

cap = cv2.VideoCapture(0)

image_count = 0  

while cap.isOpened() and image_count < 100:  # take 100 images
    ret, frame = cap.read()
    time.sleep(0.5)
    
    # picture gets taken
    #print("Cheese")
    
    # BGR 2 RGB
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Flip on horizontal
    image = cv2.flip(image, 1)
    
    # Set flag
    image.flags.writeable = False
    
    # Detections
    results = hands.process(image)
    
    # Set flag to true
    image.flags.writeable = True
    
    # RGB 2 BGR
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    
    # create copy of image (before landmarks are drawn) which can be saved later
    image_copy = image.copy()
    
    # Rendering results
    if results.multi_hand_landmarks:
        for num, hand in enumerate(results.multi_hand_landmarks):
            mp_drawing.draw_landmarks(image, hand, mp_hands.HAND_CONNECTIONS, 
                                    mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                                    mp_drawing.DrawingSpec(color=(250, 44, 250), thickness=2, circle_radius=2),
                                     )
    
    # Save the image without landmarks if recognized
    if results.multi_hand_landmarks:
        image_count += 1
        image_filename = f'{imageCategory}/image_{imageCategory}_{image_count}.jpg'
        cv2.imwrite(image_filename, image_copy)
    
    cv2.imshow('Hand Tracking', image)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


# 3. Rename And Divide Into Test, Train, Validation

In [14]:
import os
import random
import shutil

# Source and destination folder paths
source_folder =  r"C:/Users/henri/Documents/FingerCounterData/DatasetForFingerCounter/Dominik_other" # Replace this with your source folder path

training_folder = r"C:/Users/henri/Documents/FingerCounterData/DatasetForFingerCounter/Training"  # Replace this with your destination folder path
test_folder = r"C:/Users/henri/Documents/FingerCounterData/DatasetForFingerCounter/Test"  # Replace this with your destination folder path
validation_folder = r"C:/Users/henri/Documents/FingerCounterData/DatasetForFingerCounter/Validation"  # Replace this with your destination folder path

# Prefix to be added
prefix = "Dominik"

# Function to move pictures from each subfolder to the destination folder with a new prefix
def move_pictures_with_prefixes(folder_path):
    picture_files = [f for f in os.listdir(folder_path) if f.endswith(('.jpg', '.jpeg', '.png', '.gif'))]
    
    random.shuffle(picture_files)

    total_pictures = len(picture_files)
    # 80% are training data
    training_80_percent = int(total_pictures * 0.8)
    # 10% are test data
    test_10_percent = int(total_pictures * 0.1)
    # 10% are verification data
    validation_10_percent = int(total_pictures * 0.1)

    percentages = [training_80_percent, test_10_percent, validation_10_percent, total_pictures]
    folders = [training_folder, test_folder, validation_folder]
    
    for i, percentage in enumerate(percentages[:-1]):
        destination_folder = folders[i]

        selected_files = picture_files[:percentage]
        picture_files = picture_files[percentage:]

        for file_name in selected_files:
            file_path = os.path.join(folder_path, file_name)
            new_file_name = f"{prefix}_{file_name}"
            destination_path = os.path.join(destination_folder, new_file_name)
            shutil.move(file_path, destination_path)


move_pictures_with_prefixes(source_folder)


# 4. Create Landmarks And Write Them In CSV Files

In [12]:
import os
import cv2
import mediapipe as mp


# Path to the folder containing images
folder_path = r"C:\Users\leons\Documents\DeepLearning\DatasetForFingerCounter\Training"  # Replace this with the path to image folder

#path for text files
output_directory = os.path.join(folder_path, "Training Landmarks")
os.makedirs(output_directory, exist_ok=True)

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5) as hands: 
        #loop through all files in dir
        for filename in os.listdir(folder_path):
            if filename.endswith(".jpg") or filename.endswith(".png"):  # Process only image files
                file_path = os.path.join(folder_path, filename)
                frame = cv2.imread(file_path)
                
                # BGR 2 RGB
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                
                # Set flag
                image.flags.writeable = False
                
                # Detections
                results = hands.process(image)
                
                # Create a text file for each image
                output_file_path = os.path.join(output_directory, os.path.splitext(filename)[0] + "_landmarks.txt")
                with open(output_file_path, "w") as output_file:
                    output_file.write(f"{filename}\n")

                    # Detections
                    if results.multi_hand_landmarks:
                        for num, hand in enumerate(results.multi_hand_landmarks):
                            landmarks = hand.landmark
                            for landmark in landmarks:
                                output_file.write(
                                    f"{landmark.x} {landmark.y} {landmark.z}\n"
                                )

cv2.destroyAllWindows()

# ONLY FOR TEST PURPOSE: Read One Image and Create And Display Landmarks

In [11]:
import cv2
import mediapipe as mp

# Load the image file
file_path = r"C:\Users\leons\Documents\DeepLearning\DatasetForFingerCounter\Verification\Alex_image_spock_45.jpg"  # Replace this with the path to image file
frame = cv2.imread(file_path)

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5) as hands: 
   
        # BGR 2 RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Set flag
        image.flags.writeable = False
        
        # Detections
        results = hands.process(image)
        
        # Set flag to true
        image.flags.writeable = True
        
        # RGB 2 BGR
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Detections
        print(results.multi_hand_landmarks)
    
     
        # Rendering results
        if results.multi_hand_landmarks:
            for num, hand in enumerate(results.multi_hand_landmarks):
                mp_drawing.draw_landmarks(image, hand, mp_hands.HAND_CONNECTIONS, 
                                        mp_drawing.DrawingSpec(color=(20, 200, 20), thickness=2, circle_radius=4),
                                        mp_drawing.DrawingSpec(color=(20, 20, 200), thickness=2, circle_radius=2),
                                         )
        
        
        cv2.imshow('Hand Tracking', image)
        cv2.waitKey(0)

cv2.destroyAllWindows()

[landmark {
  x: 0.41592264
  y: 0.5943795
  z: 7.102046e-07
}
landmark {
  x: 0.46383724
  y: 0.58939755
  z: -0.022122784
}
landmark {
  x: 0.5100818
  y: 0.5665319
  z: -0.030141264
}
landmark {
  x: 0.5510628
  y: 0.5522729
  z: -0.03621299
}
landmark {
  x: 0.5843634
  y: 0.5553162
  z: -0.041407987
}
landmark {
  x: 0.49304077
  y: 0.47927514
  z: 0.001860652
}
landmark {
  x: 0.5193391
  y: 0.4361685
  z: 0.0045747343
}
landmark {
  x: 0.5340897
  y: 0.41092145
  z: 0.0036623036
}
landmark {
  x: 0.5467415
  y: 0.38903883
  z: 0.001101094
}
landmark {
  x: 0.46916974
  y: 0.46399274
  z: 0.0064142672
}
landmark {
  x: 0.49368724
  y: 0.41867268
  z: 0.009885185
}
landmark {
  x: 0.51285857
  y: 0.39412066
  z: 0.0065169213
}
landmark {
  x: 0.5297318
  y: 0.37296286
  z: 0.0035383205
}
landmark {
  x: 0.44118372
  y: 0.46089783
  z: 0.007314275
}
landmark {
  x: 0.4419133
  y: 0.40892377
  z: 0.005239983
}
landmark {
  x: 0.44233873
  y: 0.38143653
  z: -0.0017509328
}
landmark 